# proof of concept: шляпа
Играем в шляпу (один загадывает, остальные отгадывают)

Текущие особенности реализации:
- проверена только работа fasttext и только с помощью одним подходом к подбору слов для загадывания и отгадывания
- модели недотренированы
- тексты "грязные"
- слово "угадано", если загаданное слово составляет его часть (word in guess) -- переменная CRITERIA

In [58]:
import re
import requests
import warnings
from collections import namedtuple, defaultdict

import numpy as np
import pandas as pd
import fasttext
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display

## Обучим несколько моделей 

In [2]:
%%time

file_path = '20-newsgroups/all_texts.preprocessed.txt'

model_skipgram = fasttext.train_unsupervised(file_path, model='skipgram', dim=5)
model_cbow = fasttext.train_unsupervised(file_path, model='cbow', dim=16)
model_skipgram2 = fasttext.train_unsupervised(file_path, model='skipgram', dim=10)

CPU times: user 13min 10s, sys: 5.01 s, total: 13min 15s
Wall time: 1min 23s


In [3]:
!wc -l {file_path}

 1719263 20-newsgroups/all_texts.preprocessed.txt


In [4]:
!head {file_path}


newsgroup: sci . electronics
document_id: 52434
from: et@teal . csn . org  ( eric h .  taylor ) 
subject: re: help_with_tracking_device

in article <00969fba . e640ff10@aesop . rutgers . edu> mcdonald@aesop . rutgers . edu writes:
>[ .  .  . ]
>there are a variety of water-proof housings i could use but the real meat
>of the problem is the electronics .  .  . hence this posting .   what kind of


In [5]:
len(model_cbow.words)

76430

In [6]:
model_cbow['song']

array([-2.6425312 ,  2.040185  ,  2.256368  ,  1.049787  , -1.3677909 ,
        1.1098181 ,  0.70528245, -3.084615  , -2.828286  ,  1.7842726 ,
       -0.84591043,  0.5058342 ,  2.044178  ,  2.68207   ,  0.5996572 ,
       -0.6811738 ], dtype=float32)

In [7]:
!mkdir models
model_skipgram.save_model('models/skipgram.model')
model_skipgram2.save_model('models/skipgram2.model')
model_cbow.save_model('models/cbow.model')

mkdir: models: File exists


In [8]:
!ls -lh models

total 579968
-rw-r--r--  1 aguschin  staff   133M Sep 15 09:43 cbow.model
-rw-r--r--  1 aguschin  staff    42M Sep 15 09:43 skipgram.model
-rw-r--r--  1 aguschin  staff    83M Sep 15 09:43 skipgram2.model


## Реализации классов для игроков

In [9]:
class AbstractPlayer:
    def __init__(self):
        raise NotImplementedError()

    def explain(self, word, n_words):
        raise NotImplementedError()
        
    def guess(self, words, n_words):
        raise NotImplementedError()


if hasattr(model_skipgram2, 'get_nearest_neighbors'):
    print('using latest fasttext version')
    
    class LocalFasttextPlayer(AbstractPlayer):
        def __init__(self, model):
            self.model = model

        def find_words_for_sentence(self, sentence, n_closest):
            neighbours = self.model.get_nearest_neighbors(sentence)
            words = [word for similariry, word in neighbours][:n_closest]
            return words

        def explain(self, word, n_words):
            return self.find_words_for_sentence(word, n_words)

        def guess(self, words, n_words):
            return self.find_words_for_sentence(' '.join(words), n_words)
else:
    print('using older fasttext version')
    class LocalFasttextPlayer(AbstractPlayer):
        def __init__(self, model):
            self.model = model
            self.words = model.get_words()
            self.matrix = np.concatenate([model[word].reshape(1, -1) for word in self.words], axis=0)

        def find_words_for_vector(self, vector, n_closest):
            sims = cosine_similarity(vector.reshape(1, -1), self.matrix).ravel()
            word_sims = pd.Series(sims, index=self.model.get_words()).sort_values(ascending=False)
            return list(word_sims.head(n_closest).index)

        def find_words_for_sentence(self, sentence, n_closest):
            vector = self.model.get_sentence_vector(sentence)
            return self.find_words_for_vector(vector, n_closest)

        def explain(self, word, n_words):
            return self.find_words_for_sentence(word, n_words)

        def guess(self, words, n_words):
            return self.find_words_for_sentence(' '.join(words), n_words)


class RemotePlayer(AbstractPlayer):
    def __init__(self, url):
        self.url = url
        
    def explain(self, word, n_words):
        response = requests.get(self.url + '/explain', {'word': word, 'n_words': n_words})
        if response.status_code == 200:
            word_list = response.json()
        else:
            warnings.warn(f'request failed: {response.status_code}')
            word_list = []
        return word_list
    
    def guess(self, words, n_words):
        response = requests.get(self.url + '/guess', {'words': words, 'n_words': n_words})
        if response.status_code == 200:
            word_list = response.json()
        else:
            warnings.warn(f'request failed: {response.status_code}')
            word_list = []
        return word_list

using latest fasttext version


In [10]:
remote_player = RemotePlayer('https://obscure-everglades-02893.herokuapp.com')
# remote_player = RemotePlayer('http://127.0.0.1:5000')
print(remote_player.explain('zen', 10))
print(remote_player.guess(['zen', 'desk', 'word'], 5))

['zen', 'sin;', '>[i', 'lover', 'mad', 'rant', 'scorn', '>honestly', '*laugh*', 'forever']
['have:', '"boggs"', '>[lotsa', 'qualifier', '[now']


In [11]:
local_player = LocalFasttextPlayer(model_skipgram)
print(local_player.explain('zen', 10))
print(local_player.guess(['zen', 'desk', 'word'], 5))

['concoct', 'sin;', 'guy"', 'thine', '"de', '"he', 'god:', 'him]', 'insane', '*laugh*']
['talk-show', 'name&address', 'macpgp', 'cslipper', 'host']


## Игра

In [95]:
class Game:
    def __init__(
        self, players, words, criteria,
        n_rounds, n_explain_words, n_guessing_words,
        random_state=None
    ):
        assert len(players) >= 2
        assert criteria in ('hard', 'soft')
        self.players = players
        self.words = words
        self.criteria = criteria
        self.n_rounds = n_rounds
        self.n_explain_words = n_explain_words
        self.n_guessing_words = n_guessing_words
        
    def remove_repeated_words(self, words):
        unique_words = []
        for c in words:
            if not c in unique_words:
                unique_words.append(c)
        return unique_words
        
    def score_players(self, explainer_name, last_rounds):
        rewards = {
            player: self.n_explain_words + 1 - nround
            for player, nround in last_rounds.items()
        }
        rewards[explainer_name] = sum(rewards.values())
        return rewards

    def create_word_list(self, player, word, n_words):
        reported_words = player.explain(word, n_words)
        explain_words = reported_words[:]
        if self.criteria == 'hard':
            explain_words = explain_words[:n_words]
        explain_words = [re.sub(r'[^\w]', '', c) for c in explain_words]
        explain_words = [c for c in explain_words if word not in c]
        explain_words = self.remove_repeated_words(explain_words)
        if self.criteria == 'soft':
            explain_words = explain_words[:n_words]
        return reported_words, explain_words
    
    def check_criteria(self, word, guessed_words):
        if self.criteria == 'soft':
            guessed = any([word in c for c in guessed_words])
        else:
            guessed = word in guessed_words
        return guessed
    
    def play_round(self, explaining_player, guessing_players, word, sentence, verbose=False):
        game_round = {}
        results = {}
        if verbose:
            print(f"HOST: {sentence}")
        game_round.update({f'Explanation for "{word}" ({explaining_player.name})': sentence})
#         game_round.update({explaining_player.name: sentence})
        for player in guessing_players:
            guessed_words = player.api.guess(sentence, self.n_guessing_words)
            guessed = self.check_criteria(word, guessed_words)
            results[player.name] = guessed
            if verbose:
                print(f'GUESSING PLAYER ({player.name}) to HOST: {guessed_words}')
                print(f'HOST: {guessed}')
            game_round.update({f'Guess ({player.name})': guessed_words})
#             game_round.update({player.name: guessed_words})
        return game_round, results

    def play(self, explaining_player, guessing_players, word, criteria, verbose=False):

        if verbose:
            print(f'HOST to EXPLAINING PLAYER: the word is "{word}"')

        reported_words, guessing_by = self.create_word_list(explaining_player.api, word, self.n_explain_words)
        if verbose:
            print(f'EXPLAINING PLAYER to HOST: my wordlist is {reported_words}')
            print(f'HOST TO EXPLAINING PLAYER: cleaning your word list. Now the list is {guessing_by}')

        df = []
        success_rounds = {}
        for iround in range(1, len(guessing_by) + 1):
            if len(guessing_players) == 0:
                break
            if verbose:
                print(f'\n===ROUND {iround}===\n')
            game_round, results_round = self.play_round(
                explaining_player=explaining_player,
                guessing_players=guessing_players,
                word=word,
                sentence=guessing_by[:iround],
                verbose=verbose
            )
            for player in guessing_players[:]:
                if (player.name not in success_rounds) and results_round.get(player.name, False):
                    success_rounds[player.name] = iround
                    guessing_players = [p for p in guessing_players if p != player]
            df.append(game_round)
        
        df = pd.DataFrame(df)
        scores = self.score_players(explaining_player.name, success_rounds)
        
        return df, scores
    
    def get_words(self, complete):
        if not complete:
            words = self.words[:]
            np.random.seed(self.random_state)
            np.random.shuffle(words)
        else:
            words = []
            for word in self.words:
                words.extend([word] * len(self.players))
        return words
    
    def get_n_rounds(self, complete):
        if complete:
            return len(self.words)
        else:
            return self.n_rounds

    def run(self, verbose=0, complete=False):
        
        self.run_words = self.get_words(complete=complete)
        self.run_rounds = self.get_n_rounds(complete=complete)
            
        igame = 0
        scores = []
        scores_status = defaultdict(int)
        for r in range(self.run_rounds):
            for explaining_player in self.players:
                guessing_players = [p for p in self.players if p != explaining_player]
                word = self.run_words[igame]
                df, score = self.play(
                    explaining_player, guessing_players, word,
                    criteria=self.criteria, verbose=verbose > 1
                )
                scores.append(score)
                scores_status[(explaining_player.name, 'explaining')] += score.get(explaining_player.name, 0)
                for player in guessing_players:
                    scores_status[(player.name, 'guessing')] += score.get(player.name, 0)
                igame += 1
                if verbose > 0:
                    display(df)
                    display(score)

        self.scores = pd.DataFrame(scores).fillna(0)
        self.scores_status = pd.Series(scores_status).unstack()
        
        if verbose > 0:
            display(self.scores)
        display(self.scores.sum(axis=0))
        display(self.scores_status)

        
player = namedtuple('Player', ['name', 'api'])

N_EXPLAIN_WORDS = 10
N_GUESSING_WORDS = 5
N_ROUNDS = 1
CRITERIA = 'soft'

PLAYERS = [
    # player('skipgram team', RemotePlayer('https://obscure-everglades-02893.herokuapp.com')),
    player('skipgram team', LocalFasttextPlayer(model_skipgram)),
    player('skipgram2 team', LocalFasttextPlayer(model_skipgram2)),
    player('cbow team', LocalFasttextPlayer(model_cbow))
]

WORDS = [
    'play', 'master', 'word', 'cocoa', 'coffee',
    'september', 'jungle', 'spell', 'python', 'world',
    'cat', 'joy', 'sadness', 'small', 'stick',
]
# WORDS = ['bible']

# for vocabulary_path in [
#     'vocabulary/verbs_top_50.txt',
#     'vocabulary/nouns_top_50.txt',
#     'vocabulary/adjectives_top_50.txt'
# ]:
#     print(vocabulary_path)
#     with open(vocabulary_path) as f:
#         WORDS = f.readlines()
#         WORDS = [word.strip() for word in WORDS]

game = Game(PLAYERS, WORDS, CRITERIA, N_ROUNDS, N_EXPLAIN_WORDS, N_GUESSING_WORDS)
game.run(verbose=0, complete=True)

vocabulary/verbs_top_50.txt


cbow team         250.0
skipgram team     169.0
skipgram2 team    303.0
dtype: float64

,explaining,guessing
cbow team,98,152
skipgram team,102,67
skipgram2 team,161,142


vocabulary/nouns_top_50.txt


cbow team         233.0
skipgram team     132.0
skipgram2 team    283.0
dtype: float64

,explaining,guessing
cbow team,77,156
skipgram team,89,43
skipgram2 team,158,125


vocabulary/adjectives_top_50.txt


cbow team         217.0
skipgram team     126.0
skipgram2 team    193.0
dtype: float64

,explaining,guessing
cbow team,144,73
skipgram team,55,71
skipgram2 team,69,124
